In [7]:
import theano
import theano.tensor as T
import numpy as np
import os
import sys
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import codecs
from scipy.stats import pearsonr
import scipy

In [104]:
path_to_pretrained_vectors = 'data/glove.6B/glove.6B.200d.txt'

In [105]:
lines = [line.strip().split() for line in open(path_to_pretrained_vectors, 'r')]

In [106]:
word_to_ind = {x[0]:ind for ind, x in enumerate(lines)}
ind_to_word = {ind:x[0] for ind, x in enumerate(lines)}

In [118]:
word_to_ind['<UNK>'] = len(lines)
ind_to_word[len(lines)] = '<UNK>'

In [107]:
english_embedding = np.array([line[1:] for line in lines]).astype(np.float32)

In [108]:
english_embedding = np.vstack((english_embedding, 2 * np.random.rand(200) - 1))

In [109]:
path_to_data = 'data/train-test.hyp1-hyp2-ref'
path_to_labels = 'data/train.gold'

In [110]:
lines = [line.strip().split('|||') for line in codecs.open(path_to_data, 'rb', encoding='utf8')]

In [111]:
labels = [int(line.strip()) for line in open(path_to_labels, 'r')]

In [13]:
oov = set()
for ind, line in enumerate(lines):
    
    if ind % 1000 == 0:
        print 'Finished %d lines out of %d' % (ind, len(lines)) 
    line1 = nltk.word_tokenize(line[0].lower())
    line2 = nltk.word_tokenize(line[1].lower())
    line3 = nltk.word_tokenize(line[2].lower())
    for word in line1:
        if word not in word_to_ind:
            oov.add(word)
    for word in line2:
        if word not in word_to_ind:
            oov.add(word)
    for word in line3:
        if word not in word_to_ind:
            oov.add(word)
print 'Found %d OOV ' % (len(oov))

Finished 0 lines out of 50339
Finished 1000 lines out of 50339
Finished 2000 lines out of 50339
Finished 3000 lines out of 50339
Finished 4000 lines out of 50339
Finished 5000 lines out of 50339
Finished 6000 lines out of 50339
Finished 7000 lines out of 50339
Finished 8000 lines out of 50339
Finished 9000 lines out of 50339
Finished 10000 lines out of 50339
Finished 11000 lines out of 50339
Finished 12000 lines out of 50339
Finished 13000 lines out of 50339
Finished 14000 lines out of 50339
Finished 15000 lines out of 50339
Finished 16000 lines out of 50339
Finished 17000 lines out of 50339
Finished 18000 lines out of 50339
Finished 19000 lines out of 50339
Finished 20000 lines out of 50339
Finished 21000 lines out of 50339
Finished 22000 lines out of 50339
Finished 23000 lines out of 50339
Finished 24000 lines out of 50339
Finished 25000 lines out of 50339
Finished 26000 lines out of 50339
Finished 27000 lines out of 50339
Finished 28000 lines out of 50339
Finished 29000 lines out of

In [112]:
def apply_diag_penalty(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            matrix[i][j] *= np.exp(-0.75 * abs(float(i)/matrix.shape[0] - float(j)/matrix.shape[1]))

In [113]:
def frobenius_norm(matrix):
    return np.linalg.norm(matrix, ord='fro')

In [114]:
def assignment_problem_score(matrix):
    inds1, inds2 = scipy.optimize.linear_sum_assignment(matrix)
    return matrix[inds1, inds2].sum()

In [115]:
def reciprocal_matrix(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            matrix[i][j] = 1.0 / matrix[i][j]

In [121]:
def get_word_embedding_distance(line1, line2, line3):
    
    line1_embedding = english_embedding[[word_to_ind[word] if word in word_to_ind else word_to_ind['<UNK>'] for word in line1]]
    line2_embedding = english_embedding[[word_to_ind[word] if word in word_to_ind else word_to_ind['<UNK>'] for word in line2]]
    line3_embedding = english_embedding[[word_to_ind[word] if word in word_to_ind else word_to_ind['<UNK>'] for word in line3]]
    
    dist1 = cosine_similarity(line1_embedding, line3_embedding)
    dist2 = cosine_similarity(line2_embedding, line3_embedding)
    
    apply_diag_penalty(dist1)
    apply_diag_penalty(dist2)
    
    #score1 = frobenius_norm(dist1)
    #score2 = frobenius_norm(dist2)
    
    reciprocal_matrix(dist1)
    reciprocal_matrix(dist2)
    
    score1 = assignment_problem_score(dist1)
    score2 = assignment_problem_score(dist2)
    
    score1 = (score1 * 2) / (len(line1) + len(line3))
    score2 = (score2 * 2) / (len(line2) + len(line3))
    
    return score1, score2

In [122]:
scores = []
for ind, line in enumerate(lines):
    
    if ind % 1000 == 0:
        print 'Finished %d out of %d lines ' % (ind, len(lines))
    
    line1 = line[0].lower().split() # nltk.word_tokenize(line[0])
    line2 = line[1].lower().split() # nltk.word_tokenize(line[1])
    line3 = line[2].lower().split() # nltk.word_tokenize(line[2])
    
    scores.append(get_word_embedding_distance(line1, line2, line3))

Finished 0 out of 50339 lines 
Finished 1000 out of 50339 lines 
Finished 2000 out of 50339 lines 
Finished 3000 out of 50339 lines 
Finished 4000 out of 50339 lines 
Finished 5000 out of 50339 lines 
Finished 6000 out of 50339 lines 
Finished 7000 out of 50339 lines 
Finished 8000 out of 50339 lines 
Finished 9000 out of 50339 lines 
Finished 10000 out of 50339 lines 
Finished 11000 out of 50339 lines 
Finished 12000 out of 50339 lines 
Finished 13000 out of 50339 lines 
Finished 14000 out of 50339 lines 
Finished 15000 out of 50339 lines 
Finished 16000 out of 50339 lines 
Finished 17000 out of 50339 lines 
Finished 18000 out of 50339 lines 
Finished 19000 out of 50339 lines 
Finished 20000 out of 50339 lines 
Finished 21000 out of 50339 lines 
Finished 22000 out of 50339 lines 
Finished 23000 out of 50339 lines 
Finished 24000 out of 50339 lines 
Finished 25000 out of 50339 lines 
Finished 26000 out of 50339 lines 
Finished 27000 out of 50339 lines 
Finished 28000 out of 50339 lines

In [103]:
f = open('data/features.embedding.final', 'w')
for score in scores:
    f.write(' '.join([str(x) for x in score]) + '\n')
f.close()